In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense,
    Concatenate,
    Input,
    Embedding,
    Lambda,
    TextVectorization,
    Normalization,
    GlobalAveragePooling2D,
    GlobalAveragePooling1D,
    BatchNormalization,
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
user_df = pd.read_csv("../dataset/users.csv", delimiter=";")
artwork_df = pd.read_csv("../dataset/artworks.csv", delimiter=";")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/users.csv'

In [ ]:
user_df["place"] = user_df["place"].astype(str)
user_df["place"] = user_df["place"].fillna("N/A")

user_df["inscription_date"] = user_df["inscription_date"].astype(str)
user_df["inscription_date"] = user_df["inscription_date"].fillna("N/A")

In [ ]:
def preprocess_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image = image.convert("RGB")
        image = image.resize((224, 224))
        image_array = np.array(image)
        image_array = image_array / 255.0
        return image_array
    else:
        raise FileNotFoundError(
            f"Impossibile scaricare l'immagine dall'URL: {image_url}"
        )

In [ ]:
def create_triples(user_df, artwork_df):
    triples = []
    for _, user_row in user_df.iterrows():
        user_interactions = artwork_df[artwork_df["author"] == user_row["name"]]
        if len(user_interactions) < 2:
            continue
        Pu = user_interactions.sample(frac=1).iloc[:-1]
        i = user_interactions.sample()
        non_interacted = artwork_df[~artwork_df["author"].isin([user_row["name"]])]
        j = non_interacted.sample()
        triples.append((Pu, i, j))
    return triples

In [ ]:
def preprocess_data(triples, image_preprocessor):
    processed_triples = []

    for Pu, i, j in triples:
        Pu_images = np.array([image_preprocessor(url) for url in Pu["img"].values])
        i_image = np.array(image_preprocessor(i["img"].values[0]))
        j_image = np.array(image_preprocessor(j["img"].values[0]))
        processed_triples.append((Pu_images, i_image, j_image, Pu, i.iloc[0], j.iloc[0]))

    return processed_triples

In [ ]:
embedding_dim = 200
pu_dim = 400
max_text_words = 5000
max_comment_words = 1000
margin = 0.4

In [ ]:
name_vectorizer = TextVectorization(max_tokens=max_text_words)
name_vectorizer.adapt(user_df["name"])
place_vectorizer = TextVectorization(max_tokens=max_text_words)
place_vectorizer.adapt(user_df["place"])

inscription_date_vectorizer = TextVectorization(max_tokens=max_text_words)
inscription_date_vectorizer.adapt(user_df["inscription_date"])

art_title_vectorizer = TextVectorization(max_tokens=max_text_words)
art_title_vectorizer.adapt(artwork_df["title"])

art_author_vectorizer = TextVectorization(max_tokens=max_text_words)
art_author_vectorizer.adapt(artwork_df["author"])

art_date_vectorizer = TextVectorization(max_tokens=max_text_words)
art_date_vectorizer.adapt(artwork_df["date"])


In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False, pooling=None)
for layer in resnet.layers:
    layer.trainable = False
def extract_features(image):
    features=resnet(image)
    features = GlobalAveragePooling2D()(features)
    return features

In [ ]:
def create_text_embedding(text_input, max_words, output_dim):
    embedding = Embedding(input_dim=max_words, output_dim=output_dim)(text_input)
    pooling_embedding = GlobalAveragePooling1D()(embedding)
    return pooling_embedding

In [ ]:
normalization_layer = Normalization()

In [ ]:
input_user_name = Input(shape=(1,), name="input_user_name")
input_user_place = Input(shape=(1,), name="input_user_place")
input_user_inscription_date = Input(shape=(1,), name="input_user_inscription_date")
input_user_page_views = Input(shape=(1,), name="input_user_page_views")
input_user_followers = Input(shape=(1,), name="input_user_followers")
input_user_follow = Input(shape=(1,), name="input_user_follow")
input_user_favourites = Input(shape=(1,), name="input_user_favourites")
input_user_comments_made = Input(shape=(1,), name="input_user_comments_made")
input_user_comments_received = Input(shape=(1,), name="input_user_comments_received")

normalized_inscription_date = normalization_layer(input_user_inscription_date)
normalized_page_views = normalization_layer(input_user_page_views)
normalized_followers = normalization_layer(input_user_followers)
normalized_follow = normalization_layer(input_user_follow)
normalized_favourites = normalization_layer(input_user_favourites)
normalized_comments_made = normalization_layer(input_user_comments_made)
normalized_comments_received = normalization_layer(input_user_comments_received)

normalized_inscription_date = Dense(embedding_dim, activation='selu')(normalized_inscription_date)
normalized_page_views = Dense(embedding_dim, activation="selu")(normalized_page_views)
normalized_followers = Dense(embedding_dim, activation="selu")(normalized_followers)
normalized_follow = Dense(embedding_dim, activation="selu")(normalized_follow)
normalized_favourites = Dense(embedding_dim, activation="selu")(normalized_favourites)
normalized_comments_made = Dense(embedding_dim, activation="selu")(
    normalized_comments_made
)
normalized_comments_received = Dense(embedding_dim, activation="selu")(
    normalized_comments_received
)

user_features = Concatenate()(
    [
        create_text_embedding(input_user_name, max_text_words, embedding_dim),
        create_text_embedding(input_user_place, max_text_words, embedding_dim),
        create_text_embedding(input_user_inscription_date, max_text_words, embedding_dim),
        normalized_page_views,
        normalized_followers,
        normalized_follow,
        normalized_favourites,
        normalized_comments_made,
        normalized_comments_received,
    ]
)

In [ ]:
input_art_title = Input(shape=(8,), name="input_art_title")
input_art_author = Input(shape=(1,), name="input_art_author")
input_art_likes = Input(shape=(1,), name="input_art_likes")
input_art_views = Input(shape=(1,), name="input_art_views")
input_art_date = Input(shape=(1,), name="input_art_date")

art_title_embedding = create_text_embedding(
    input_art_title, max_text_words, embedding_dim
)
art_author_embedding = create_text_embedding(
    input_art_author, max_text_words, embedding_dim
)
art_date_embedding = create_text_embedding(
    input_art_date, max_text_words, embedding_dim
)

art_features = Concatenate()(
    [
        art_title_embedding,
        art_author_embedding,
        normalization_layer(input_art_likes),
        normalization_layer(input_art_views),
        normalization_layer(input_art_date),
    ]
)

In [ ]:
def custom_reduce_sum(x, y):
    return K.sum(x * y, axis=1)

In [ ]:
def custom_triplet_loss(y_true, y_pred, margin=0.2):
    score_i, score_j = tf.split(y_pred, num_or_size_splits=2, axis=-1)
    loss = tf.maximum(0.0, margin + score_j - score_i)
    return tf.reduce_mean(loss)

In [ ]:
input_pu = Input(shape=(3,224, 224, 3), name="input_pu")
input_i = Input(shape=(224, 224, 3), name="input_i")
input_j = Input(shape=(224, 224, 3), name="input_j")
pu_features = Lambda(
    lambda x: K.map_fn(
        lambda y: extract_features(y), x
    ),
    output_shape=(None, embedding_dim),
)(input_pu)

i_features = extract_features(input_i)
j_features = extract_features(input_j)

dense_layer_1 = Dense(embedding_dim, activation="selu", name="dense_layer_1")
dense_layer_2 = Dense(embedding_dim, activation="selu", name="dense_layer_2")

reduced_pu = Lambda(
    lambda x: K.map_fn(
        lambda y: dense_layer_2(dense_layer_1(y)), x
    ),
    output_shape=(None, embedding_dim),
)(pu_features)


reduced_i = dense_layer_2(dense_layer_1(i_features))
reduced_j = dense_layer_2(dense_layer_1(j_features))

concat_i= Concatenate()([reduced_i, user_features, art_features])
concat_j= Concatenate()([reduced_j, user_features, art_features])


dense_comb_i= Dense(embedding_dim, activation="selu", name="dense_comb_i")(concat_i)
dense_comb_j= Dense(embedding_dim, activation="selu", name="dense_comb_j")(concat_j)


average_pooled_pu = Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)
max_pooled_pu = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim,))(
    reduced_pu
)

pooled_pu = Concatenate()([average_pooled_pu, max_pooled_pu])


pu_dense_1 = Dense(300, activation="selu", name="pu_dense_1")(pooled_pu)
pu_dense_2 = Dense(200, activation="selu", name="pu_dense_2")(pu_dense_1)

final_pu = Dense(200, activation="selu", name="pu_dense_3")(pu_dense_2)


In [ ]:
score_i = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))([final_pu, dense_comb_i])
score_j = Lambda(lambda x: K.sum(x[0] * x[1], axis=1, keepdims=True))([final_pu, dense_comb_j])
output_scores = Concatenate(axis=-1)([score_i, score_j])

In [ ]:

def build_curatornet_model():
    tf.keras.backend.clear_session()
    curatornet = Model(
        inputs=[
            input_user_name,
            input_user_place,
            input_user_inscription_date,
            input_user_page_views,
            input_user_followers,
            input_user_follow,
            input_user_favourites,
            input_user_comments_made,
            input_user_comments_received,
            input_art_title,
            input_art_author,
            input_art_likes,
            input_art_views,
            input_art_date,
            input_pu,
            input_i,
            input_j,
        ],
        outputs=output_scores,
    )
    return curatornet

# curatornet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001), loss=custom_triplet_loss)
# curatornet.summary()

In [ ]:
def prepare_inputs(processed_triples, users_df, target_image_count=3, expected_length=183):
    inputs = {
        "input_user_name": [],
        "input_user_place": [],
        "input_user_inscription_date": [],
        "input_user_page_views": [],
        "input_user_followers": [],
        "input_user_follow": [],
        "input_user_favourites": [],
        "input_user_comments_made": [],
        "input_user_comments_received": [],
        "input_art_title": [],
        "input_art_author": [],
        "input_art_likes": [],
        "input_art_views": [],
        "input_art_date": [],
        "input_pu": [],
        "input_i": [],
        "input_j": [],
    }

    for pu_images, i_image, j_image, Pu_meta, i_meta, j_meta in processed_triples:
        user_features = users_df[users_df["name"] == i_meta["author"]].iloc[0]
        if pu_images.shape[0] < target_image_count:
            pad_width = target_image_count - pu_images.shape[0]
            pu_images = np.pad(pu_images, ((0, pad_width), (0, 0), (0, 0), (0, 0)), mode='constant')
        elif pu_images.shape[0] > target_image_count:
            pu_images = pu_images[:target_image_count]

        inputs["input_pu"].append(pu_images)
        inputs["input_i"].append(i_image)
        inputs["input_j"].append(j_image)
        inputs["input_user_name"].append(user_features["name"])
        inputs["input_user_place"].append(user_features["place"])
        inputs["input_user_inscription_date"].append(user_features["inscription_date"])
        inputs["input_user_page_views"].append(user_features["number_page_views"])
        inputs["input_user_followers"].append(user_features["number_followers"])
        inputs["input_user_follow"].append(user_features["number_follow"])
        inputs["input_user_favourites"].append(user_features["number_favourites"])
        inputs["input_user_comments_made"].append(user_features["number_comments_made"])
        inputs["input_user_comments_received"].append(
            user_features["number_comments_receveid"]
        )
        inputs["input_art_title"].append(i_meta["title"])
        inputs["input_art_author"].append(i_meta["author"])
        inputs["input_art_likes"].append(i_meta["likes"])
        inputs["input_art_views"].append(i_meta["number_of_views"])
        inputs["input_art_date"].append(i_meta["date"])

    for key in inputs:
        try:
            if not isinstance(inputs[key], np.ndarray):
                inputs[key] = np.array(inputs[key])

            current_length = len(inputs[key])
            if current_length != expected_length:
                print(f"Adjusting {key} from {current_length} to {expected_length}")
                repeat_factor = max(1, expected_length // current_length)
                if repeat_factor > 1:
                    inputs[key] = np.tile(inputs[key], (repeat_factor,) + (1,) * (inputs[key].ndim - 1))
                if len(inputs[key]) > expected_length:
                    inputs[key] = inputs[key][:expected_length]
        except Exception as e:
            print(f"Warning: Could not convert {key} to numpy array: {e}")
    return inputs

In [ ]:
def vectorizer_input(inputs):
  inputs["input_user_name"] = name_vectorizer(inputs["input_user_name"])
  inputs["input_user_place"] = place_vectorizer(inputs["input_user_place"])
  inputs["input_user_inscription_date"] = inscription_date_vectorizer(inputs["input_user_inscription_date"])
  inputs["input_art_title"] = art_title_vectorizer(inputs["input_art_title"])
  inputs["input_art_author"] = art_author_vectorizer(inputs["input_art_author"])
  inputs["input_art_date"] = art_date_vectorizer(inputs["input_art_date"])
  return inputs


In [ ]:
def expand_inputs(inputs):
  inputs["input_user_page_views"] = np.expand_dims(inputs["input_user_page_views"], axis=-1)
  inputs["input_user_followers"] = np.expand_dims(inputs["input_user_followers"], axis=-1)
  inputs["input_user_follow"] = np.expand_dims(inputs["input_user_follow"], axis=-1)
  inputs["input_user_favourites"] = np.expand_dims(inputs["input_user_favourites"], axis=-1)
  inputs["input_user_comments_made"] = np.expand_dims(inputs["input_user_comments_made"], axis=-1)
  inputs["input_user_comments_received"] = np.expand_dims(inputs["input_user_comments_received"], axis=-1)
  inputs["input_art_likes"] = np.expand_dims(inputs["input_art_likes"], axis=-1)
  inputs["input_art_views"] = np.expand_dims(inputs["input_art_views"], axis=-1)
  return inputs

In [ ]:
def replace_nans_in_inputs(inputs):
    for key, value in inputs.items():
        if isinstance(value, np.ndarray):
            value = tf.convert_to_tensor(value)
        if value.dtype.is_floating:
            if tf.math.reduce_any(tf.math.is_nan(value)):
                inputs[key] = tf.where(tf.math.is_nan(value), tf.zeros_like(value), value)
            else:
                print(f"Nessun NaN in {key}")
        else:
            print(f"{key} non è un tipo a virgola mobile, saltato.")       

In [ ]:
def convert_inputs_to_tensors(inputs):
    for key, value in inputs.items():
        if isinstance(value, np.ndarray) or isinstance(value, list):
            inputs[key] = tf.convert_to_tensor(value)
    return inputs

In [ ]:
def check_input_types(inputs):
    for key, value in inputs.items():
        print(f"Input key: {key}")
        print(f" - Type: {type(value)}")
        if isinstance(value, np.ndarray):
            print(f" - Dtype: {value.dtype}")
            print(f" - Shape: {value.shape}")
        else:
            print(f" - Not a NumPy array")
        print()

In [ ]:
triples = create_triples(user_df, artwork_df)
preprocessed_triples = preprocess_data(triples, preprocess_image)
inputs = prepare_inputs(preprocessed_triples, user_df)
inputs = vectorizer_input(inputs)
inputs = expand_inputs(inputs)
convert_inputs_to_tensors(inputs)
replace_nans_in_inputs(inputs)

input_user_name non è un tipo a virgola mobile, saltato.
input_user_place non è un tipo a virgola mobile, saltato.
input_user_inscription_date non è un tipo a virgola mobile, saltato.
input_art_title non è un tipo a virgola mobile, saltato.
input_art_author non è un tipo a virgola mobile, saltato.
input_art_likes non è un tipo a virgola mobile, saltato.
input_art_views non è un tipo a virgola mobile, saltato.
input_art_date non è un tipo a virgola mobile, saltato.
Nessun NaN in input_pu
Nessun NaN in input_i
Nessun NaN in input_j


In [ ]:
n_splits = 5 
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
y = np.zeros((len(triples), 1))
fold = 1
val_losses = []
val_accuracies = []
for train_index, test_index in kf.split(inputs["input_user_name"]):
    print(f"Fold {fold}")
    print(f"Train Index: {train_index}")    
    print(f"Test Index: {test_index}")
    X_train = {key: np.array(val)[train_index] for key, val in inputs.items()}
    X_test = {key: np.array(val)[test_index] for key, val in inputs.items()}
    y_train, y_test = y[train_index], y[test_index]
    
    print(f"divisione completata")

    model =  build_curatornet_model()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.00000005),
        loss=custom_triplet_loss,
        metrics=["accuracy"],
    )

    print(f"compilazione completata")

    model.fit(
        X_train, y_train, epochs=10, verbose=0
    )

    print(f"addestramento completato")

    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)

    print(
        f"Fold {fold} - Validation Loss: {val_loss} - Validation Accuracy: {val_accuracy}"
    )

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    fold += 1

average_val_loss = np.mean(val_losses)
print(
    f"Mean Validation Loss after {n_splits}-Fold Cross Validation: {average_val_loss}"
)

Fold 1
Train Index: [  0   1   2   3   4   5   6   7   8  10  11  12  13  14  17  20  21  22
  23  25  26  27  28  31  32  33  34  36  37  38  39  40  41  43  44  46
  47  48  49  50  52  53  54  57  58  59  61  62  63  64  65  67  70  71
  72  73  74  76  77  78  79  80  81  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  99 100 101 102 103 104 105 106 107 108 109 110 112
 114 115 116 118 120 121 122 123 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 148 149 151 152 153 154 156 157 158
 159 160 162 164 165 166 167 168 169 170 171 172 175 176 177 178 179 180
 181 182]
Test Index: [  9  15  16  18  19  24  29  30  35  42  45  51  55  56  60  66  68  69
  75  82  97  98 111 113 117 119 124 125 145 146 147 150 155 161 163 173
 174]
divisione completata
compilazione completata
